## Detectability

In [2]:
import json
import pandas as pd

### Load data

In [3]:
def load_data(path: str):
    with open(path) as file:
        json_data = json.load(file)
    df = pd.DataFrame(json_data)

    if "_id" in df.columns:
        df = df.drop(columns="_id")

    return df

unobfuscated_unoptimized = load_data("./data/unoptimized/unobfuscated.json")
unobfuscated_optimized = load_data("./data/optimized/unobfuscated.json")

tigress_unoptimized = load_data("./data/unoptimized/tigress.json")
tigress_optimized = load_data("./data/optimized/tigress.json")

llvm_unoptimized = load_data("./data/unoptimized/llvm.json")
llvm_optimized = load_data("./data/optimized/llvm.json")

wasm_mutate_unoptimized = load_data("./data/unoptimized/wasm-mutate.json")
wasm_mutate_optimized = load_data("./data/optimized/wasm-mutate.json")

wasm_mutate_unoptimized['transformation'] = wasm_mutate_unoptimized['mutator']
wasm_mutate_optimized['transformation'] = wasm_mutate_optimized['mutator']

In [4]:
transformation_label_map = {
    "flatten": "Flattening",
    "randomfuns": "Random Functions",
    "encodearithmetic": "Encode Arithmetic",
    "encodeliterals": "Encode Literals",
    "split": "Function Splitting",
    "virtualize": "Virtualization",
    "antitaintanalysis": "Anti Taint Analysis",
    "antialiasanalysis": "Anti Alias Analysis"
}

# Create a new column 'transformation_label' based on the 'transformation' column values
tigress_unoptimized['transformation'] = tigress_unoptimized['transformation'].map(transformation_label_map)
tigress_optimized['transformation'] = tigress_optimized['transformation'].map(transformation_label_map)

# Create a new column 'transformation_label' based on the 'transformation' column values
transformation_label_map = {
    "cffobf": "Control Flow Flattening",
    "bcfobf": "Bogus Control Flow",
    "constenc": "Constants Encryption",
    "strcry": "String Encryption",
    "splitobf": "Basic Block Splitting",
    "indibran": "Indirect Branches",
    "subobf": "Substitute Instruction",
    "funcwra": "Function Wrapper"
}
llvm_unoptimized['transformation'] = llvm_unoptimized['transformation'].map(transformation_label_map)
llvm_optimized['transformation'] = llvm_optimized['transformation'].map(transformation_label_map)

# Create a new column 'transformation_label' based on the 'mutator' column values
wasm_mutate_unoptimized['transformation'] = wasm_mutate_unoptimized['mutator']
wasm_mutate_optimized['transformation'] = wasm_mutate_optimized['mutator']
transformation_label_map = {
    "AddCustomSectionMutator": "Add Custom Section",
    "PeepholeMutator": "Peephole",
    "CodemotionMutator": "Code Motion",
    "AddTypeMutator": "Add Type",
    "AddFunctionMutator": "Add Function",
    "RemoveSection": "Remove Section",
    "RemoveItemMutator": "Remove Item",
    "ReorderCustomSectionMutator": "Reorder Custom Section", 
    "CustomSectionMutator": "Custom Section"
}
wasm_mutate_unoptimized['transformation'] = wasm_mutate_unoptimized['transformation'].map(transformation_label_map)
wasm_mutate_optimized['transformation'] = wasm_mutate_optimized['transformation'].map(transformation_label_map)

### Extract detection results

In [5]:
def extract_detection_results(df, optimization, obfuscation=None):
    # Extracting the minos, miner_ray, virustotal, and wasim result values
    df["minos_result"] = df["minos"].apply(lambda x: x["result"])
    df["miner_ray_result"] = df["miner_ray"].apply(lambda x: x["result"])
    df["virustotal_result"] = df["virustotal"].apply(lambda x: x["result"])

    # Extracting wasim results
    def extract_wasim_cryptominer(model_results):
        model_results = {k: v for k, v in model_results.items() if k != "code"}
        sorted_results = sorted(model_results.items(), key=lambda x: x[1], reverse=True)
        return int(sorted_results[0][0] == "Cryptominer")

    df["wasim_neural"] = df["wasim"].apply(lambda x: extract_wasim_cryptominer(x["result"]["neural"]))
    df["wasim_random"] = df["wasim"].apply(lambda x: extract_wasim_cryptominer(x["result"]["random"]))
    df["wasim_svm"] = df["wasim"].apply(lambda x: extract_wasim_cryptominer(x["result"]["svm"]))
    df["wasim_naive"] = df["wasim"].apply(lambda x: extract_wasim_cryptominer(x["result"]["naive"]))

    df["ground_truth"] = df["category"].apply(lambda x: 1 if x == "miners" else 0)


    # Check if 'transformation' column exists
    if "transformation" in df.columns:
        transformation_column = df["transformation"]
    else:
        transformation_column = [None] * len(df)

    # Creating the new DataFrame with the specified columns
    detection_results_df = pd.DataFrame(
        {
            "obfuscation": [obfuscation] * len(df),
            "optimization": [optimization] * len(df),
            "transformation": transformation_column,
            "name": df["name"],
            "ground_truth": df["ground_truth"],
            "minos": df["minos_result"],
            "miner_ray": df["miner_ray_result"],
            "virustotal": df["virustotal_result"],
            "wasim_neural": df["wasim_neural"],
            "wasim_random": df["wasim_random"],
            "wasim_svm": df["wasim_svm"],
            "wasim_naive": df["wasim_naive"],
        }
    )

    return detection_results_df

#### Combine into one dataframe

In [6]:
unobfuscated_unoptimized_results = extract_detection_results(unobfuscated_unoptimized, optimization="unoptimized")
unobfuscated_optimized_results = extract_detection_results(unobfuscated_optimized, optimization="optimized")

tigress_unoptimized_results = extract_detection_results(tigress_unoptimized, optimization="unoptimized", obfuscation="Tigress")
tigress_optimized_results = extract_detection_results(tigress_optimized, optimization="optimized", obfuscation="Tigress")

llvm_unoptimized_results = extract_detection_results(llvm_unoptimized, optimization="unoptimized", obfuscation="llvm")
llvm_optimized_results = extract_detection_results(llvm_optimized, optimization="optimized", obfuscation="llvm")

filtered_wasm_mutate_data_unoptimized = wasm_mutate_unoptimized[wasm_mutate_unoptimized['miner_ray'].notna()]
filtered_wasm_mutate_data_optimized = wasm_mutate_optimized[wasm_mutate_optimized['miner_ray'].notna()]

filtered_wasm_mutate_data_unoptimized["transformation"] = filtered_wasm_mutate_data_unoptimized.apply(lambda row: f"Iteration {row['iteration']}" if row['iteration'] % 100 == 0 else row['transformation'], axis=1)
filtered_wasm_mutate_data_optimized["transformation"] = filtered_wasm_mutate_data_optimized.apply(lambda row: f"Iteration {row['iteration']}" if row['iteration'] % 100 == 0 else row['transformation'], axis=1)

wasm_mutate_unoptimized_results = extract_detection_results(filtered_wasm_mutate_data_unoptimized, optimization="unoptimized", obfuscation="wasm-mutate")
wasm_mutate_optimized_results = extract_detection_results(filtered_wasm_mutate_data_optimized, optimization="optimized", obfuscation="wasm-mutate")

df = pd.concat([unobfuscated_unoptimized_results, unobfuscated_optimized_results, tigress_unoptimized_results, tigress_optimized_results, llvm_unoptimized_results, llvm_optimized_results, wasm_mutate_unoptimized_results, wasm_mutate_optimized_results], ignore_index=True)

/tmp/ipykernel_3519/1314757861.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_wasm_mutate_data_unoptimized["transformation"] = filtered_wasm_mutate_data_unoptimized.apply(lambda row: f"Iteration {row['iteration']}" if row['iteration'] % 100 == 0 else row['transformation'], axis=1)
/tmp/ipykernel_3519/1314757861.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_wasm_mutate_data_optimized["transformation"] = filtered_wasm_mutate_data_optimized.apply(lambda row: f"Iteration {row['ite

#### Calculate performance of detection methods

In [7]:
def evaluate_method(df, method):
    
    # Calculate true positive (TP), false positive (FP), false negative (FN), and true negative (TN)
    TP = sum((df[method] == 1) & (df["ground_truth"] == 1))
    FP = sum((df[method] == 1) & (df["ground_truth"] == 0))
    FN = sum((df[method] == 0) & (df["ground_truth"] == 1))
    TN = sum((df[method] == 0) & (df["ground_truth"] == 0))
    
    # Calculate precision, recall, and F1-score
    precision = round(TP / (TP + FP) if TP + FP > 0 else 0, 2)
    recall = round(TP / (TP + FN) if TP + FN > 0 else 0, 2)
    f1_score = round((2 * precision * recall) / (precision + recall) if precision + recall > 0 else 0, 2)
    
    # Calculate Matthews Correlation Coefficient (MCC)
    mcc_numerator = (TP * TN) - (FP * FN)
    mcc_denominator = ((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN)) ** 0.5
    mcc = round(mcc_numerator / mcc_denominator if mcc_denominator > 0 else 0, 2)
    
    return precision, recall, f1_score, mcc



def calculate_metrics_by_optimization(df, method):
    # Split the data into unoptimized and optimized subsets
    unoptimized_df = df[df["optimization"] == "unoptimized"]
    optimized_df = df[df["optimization"] == "optimized"]

    # Calculate the evaluation metrics for each subset
    unoptimized_metrics = evaluate_method(unoptimized_df, method)
    optimized_metrics = evaluate_method(optimized_df, method)

    # Create a new DataFrame with the evaluation metrics
    metrics_df = pd.DataFrame(
        {
            "obfuscation": [None],
            "transformation": [None],
            f"unoptimized_{method}_precision": [unoptimized_metrics[0]],
            f"unoptimized_{method}_recall": [unoptimized_metrics[1]],
            f"unoptimized_{method}_f1_score": [unoptimized_metrics[2]],
            f"optimized_{method}_precision": [optimized_metrics[0]],
            f"optimized_{method}_recall": [optimized_metrics[1]],
            f"optimized_{method}_f1_score": [optimized_metrics[2]],
            f"diff_{method}_f1_score": [optimized_metrics[2] - unoptimized_metrics[2]],
        }
    )

    return metrics_df


def calculate_metrics_by_obfuscation_and_optimization(df, method):
    # Find unique obfuscation methods and transformations
    unique_obfuscations = df["obfuscation"].dropna().unique()
    unique_transformations = df["transformation"].dropna().unique()
    
    metrics_df_list = []
    
    # Handle the case without obfuscation
    no_obfuscation_df = df[df["obfuscation"].isna()]
    no_obfuscation_metrics = calculate_metrics_by_optimization(no_obfuscation_df, method)
    metrics_df_list.append(no_obfuscation_metrics)

    # Iterate over obfuscation methods and transformations, calculating metrics for each combination
    for obfuscation in unique_obfuscations:
        for transformation in unique_transformations:
            filtered_df = df[(df["obfuscation"] == obfuscation) & (df["transformation"] == transformation)]
            if len(filtered_df) > 0:
                obfuscation_transformation_metrics = calculate_metrics_by_optimization(filtered_df, method)
                obfuscation_transformation_metrics["obfuscation"] = obfuscation
                obfuscation_transformation_metrics["transformation"] = transformation
                metrics_df_list.append(obfuscation_transformation_metrics)
    
    # Concatenate all DataFrames in the list
    metrics_df = pd.concat(metrics_df_list, ignore_index=True)
    return metrics_df

In [23]:
# Calculate metrics for the 'minos' method
minos_metrics = calculate_metrics_by_obfuscation_and_optimization(df, "minos")
latex_table = minos_metrics.to_latex(index=False)
minos_metrics

/tmp/ipykernel_3519/3817004875.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = minos_metrics.to_latex(index=False)


,obfuscation,transformation,unoptimized_minos_precision,unoptimized_minos_recall,unoptimized_minos_f1_score,optimized_minos_precision,optimized_minos_recall,optimized_minos_f1_score,diff_minos_f1_score
0,None,None,1.000000,1.0,1.000000,0.888889,0.8,0.842105,-1.578947e-01
1,Tigress,Encode Arithmetic,0.583333,0.7,0.636364,0.750000,0.9,0.818182,1.818182e-01
2,Tigress,Function Splitting,0.400000,0.2,0.266667,0.800000,0.4,0.533333,2.666667e-01
3,Tigress,Random Functions,0.714286,1.0,0.833333,0.769231,1.0,0.869565,3.623188e-02
4,Tigress,Flattening,0.666667,1.0,0.800000,0.727273,0.8,0.761905,-3.809524e-02
5,Tigress,Encode Literals,0.555556,1.0,0.714286,0.666667,0.8,0.727273,1.298701e-02
6,Tigress,Anti Taint Analysis,0.769231,1.0,0.869565,0.727273,0.8,0.761905,-1.076605e-01
7,Tigress,Anti Alias Analysis,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000e+00
8,Tigress,Virtualization,0.750000,0.9,0.818182,1.000000,0.8,0.888889,7.070707e-02
9,llvm,Control Flow Flattening,0.666667,1.0,0.800000,0.666667,1.0,0.800000,0.000000e+00


In [9]:
wasim_neural = calculate_metrics_by_obfuscation_and_optimization(df, "wasim_neural")
wasim_neural

,obfuscation,transformation,unoptimized_wasim_neural_precision,unoptimized_wasim_neural_recall,unoptimized_wasim_neural_f1_score,optimized_wasim_neural_precision,optimized_wasim_neural_recall,optimized_wasim_neural_f1_score,diff_wasim_neural_f1_score
0,None,None,1.00,0.2,0.33,0.00,0.0,0.00,-0.33
1,Tigress,Encode Arithmetic,0.78,0.7,0.74,0.90,0.9,0.90,0.16
2,Tigress,Function Splitting,0.91,1.0,0.95,1.00,0.6,0.75,-0.20
3,Tigress,Random Functions,0.91,1.0,0.95,0.86,0.6,0.71,-0.24
4,Tigress,Flattening,0.00,0.0,0.00,0.00,0.0,0.00,0.00
5,Tigress,Encode Literals,0.83,1.0,0.91,0.91,1.0,0.95,0.04
6,Tigress,Anti Taint Analysis,0.89,0.8,0.84,0.91,1.0,0.95,0.11
7,Tigress,Anti Alias Analysis,0.00,0.0,0.00,1.00,1.0,1.00,1.00
8,Tigress,Virtualization,0.00,0.0,0.00,0.00,0.0,0.00,0.00
9,llvm,Control Flow Flattening,0.00,0.0,0.00,0.00,0.0,0.00,0.00


In [10]:
wasim_naive = calculate_metrics_by_obfuscation_and_optimization(df, "wasim_naive")
wasim_naive

,obfuscation,transformation,unoptimized_wasim_naive_precision,unoptimized_wasim_naive_recall,unoptimized_wasim_naive_f1_score,optimized_wasim_naive_precision,optimized_wasim_naive_recall,optimized_wasim_naive_f1_score,diff_wasim_naive_f1_score
0,None,None,1.00,0.1,0.18,0.00,0.0,0.00,-0.18
1,Tigress,Encode Arithmetic,0.00,0.0,0.00,0.00,0.0,0.00,0.00
2,Tigress,Function Splitting,0.50,0.1,0.17,0.00,0.0,0.00,-0.17
3,Tigress,Random Functions,1.00,0.1,0.18,0.00,0.0,0.00,-0.18
4,Tigress,Flattening,0.00,0.0,0.00,0.50,0.1,0.17,0.17
5,Tigress,Encode Literals,0.50,0.1,0.17,0.50,0.1,0.17,0.00
6,Tigress,Anti Taint Analysis,0.00,0.0,0.00,0.00,0.0,0.00,0.00
7,Tigress,Anti Alias Analysis,0.00,0.0,0.00,0.50,0.1,0.17,0.17
8,Tigress,Virtualization,1.00,0.1,0.18,1.00,0.1,0.18,0.00
9,llvm,Control Flow Flattening,1.00,0.1,0.18,0.00,0.0,0.00,-0.18


In [11]:
wasim_random = calculate_metrics_by_obfuscation_and_optimization(df, "wasim_random")
wasim_random

,obfuscation,transformation,unoptimized_wasim_random_precision,unoptimized_wasim_random_recall,unoptimized_wasim_random_f1_score,optimized_wasim_random_precision,optimized_wasim_random_recall,optimized_wasim_random_f1_score,diff_wasim_random_f1_score
0,None,None,0.0,0.0,0,0.67,1.0,0.80,0.80
1,Tigress,Encode Arithmetic,0.0,0.0,0,0.67,1.0,0.80,0.80
2,Tigress,Function Splitting,0.0,0.0,0,0.67,1.0,0.80,0.80
3,Tigress,Random Functions,0.0,0.0,0,0.67,1.0,0.80,0.80
4,Tigress,Flattening,0.0,0.0,0,0.77,1.0,0.87,0.87
5,Tigress,Encode Literals,0.0,0.0,0,0.62,1.0,0.77,0.77
6,Tigress,Anti Taint Analysis,0.0,0.0,0,0.67,1.0,0.80,0.80
7,Tigress,Anti Alias Analysis,0.0,0.0,0,0.00,0.0,0.00,0.00
8,Tigress,Virtualization,0.0,0.0,0,0.83,1.0,0.91,0.91
9,llvm,Control Flow Flattening,0.0,0.0,0,0.71,1.0,0.83,0.83


In [12]:
wasim_svm = calculate_metrics_by_obfuscation_and_optimization(df, "wasim_svm")
wasim_svm

,obfuscation,transformation,unoptimized_wasim_svm_precision,unoptimized_wasim_svm_recall,unoptimized_wasim_svm_f1_score,optimized_wasim_svm_precision,optimized_wasim_svm_recall,optimized_wasim_svm_f1_score,diff_wasim_svm_f1_score
0,None,None,0,0.0,0,0,0.0,0,0
1,Tigress,Encode Arithmetic,0,0.0,0,0,0.0,0,0
2,Tigress,Function Splitting,0,0.0,0,0,0.0,0,0
3,Tigress,Random Functions,0,0.0,0,0,0.0,0,0
4,Tigress,Flattening,0,0.0,0,0,0.0,0,0
5,Tigress,Encode Literals,0,0.0,0,0,0.0,0,0
6,Tigress,Anti Taint Analysis,0,0.0,0,0,0.0,0,0
7,Tigress,Anti Alias Analysis,0,0.0,0,0,0.0,0,0
8,Tigress,Virtualization,0,0.0,0,0,0.0,0,0
9,llvm,Control Flow Flattening,0,0.0,0,0,0.0,0,0


In [13]:
virustotal_metrics = calculate_metrics_by_obfuscation_and_optimization(df, "virustotal")
virustotal_metrics

,obfuscation,transformation,unoptimized_virustotal_precision,unoptimized_virustotal_recall,unoptimized_virustotal_f1_score,optimized_virustotal_precision,optimized_virustotal_recall,optimized_virustotal_f1_score,diff_virustotal_f1_score
0,None,None,0,0.0,0,0,0.0,0,0
1,Tigress,Encode Arithmetic,0,0.0,0,0,0.0,0,0
2,Tigress,Function Splitting,0,0.0,0,0,0.0,0,0
3,Tigress,Random Functions,0,0.0,0,0,0.0,0,0
4,Tigress,Flattening,0,0.0,0,0,0.0,0,0
5,Tigress,Encode Literals,0,0.0,0,0,0.0,0,0
6,Tigress,Anti Taint Analysis,0,0.0,0,0,0.0,0,0
7,Tigress,Anti Alias Analysis,0,0.0,0,0,0.0,0,0
8,Tigress,Virtualization,0,0.0,0,0,0.0,0,0
9,llvm,Control Flow Flattening,0,0.0,0,0,0.0,0,0


In [14]:
miner_ray_metrics = calculate_metrics_by_obfuscation_and_optimization(df, "miner_ray")
miner_ray_metrics

,obfuscation,transformation,unoptimized_miner_ray_precision,unoptimized_miner_ray_recall,unoptimized_miner_ray_f1_score,optimized_miner_ray_precision,optimized_miner_ray_recall,optimized_miner_ray_f1_score,diff_miner_ray_f1_score
0,None,None,0,0.0,0,0,0.0,0,0
1,Tigress,Encode Arithmetic,0,0.0,0,0,0.0,0,0
2,Tigress,Function Splitting,0,0.0,0,0,0.0,0,0
3,Tigress,Random Functions,0,0.0,0,0,0.0,0,0
4,Tigress,Flattening,0,0.0,0,0,0.0,0,0
5,Tigress,Encode Literals,0,0.0,0,0,0.0,0,0
6,Tigress,Anti Taint Analysis,0,0.0,0,0,0.0,0,0
7,Tigress,Anti Alias Analysis,0,0.0,0,0,0.0,0,0
8,Tigress,Virtualization,0,0.0,0,0,0.0,0,0
9,llvm,Control Flow Flattening,0,0.0,0,0,0.0,0,0
